In [2]:
import pandas as pd

Importing Datasets

In [3]:
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
tags = pd.read_csv('/content/tags.csv')
links = pd.read_csv('/content/links.csv')


In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [7]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [9]:
# remove special character
movies.columns = movies.columns.str.replace(' ', '')

In [11]:
ratings.columns = ratings.columns.str.replace(' ','')

**Without normalizing data**

In [14]:
ratings_count = ratings.groupby('movieId')['rating'].agg(['mean','count'])

weight_rating_count = 0.25
weight_rating_average = 0.75

ratings_count ['popularity_score'] = (
    weight_rating_count * ratings_count['mean'] +
    weight_rating_average * ratings_count['count']
)
popular_movies =ratings_count.sort_values(by = 'popularity_score',ascending = False).reset_index()

#For top 10
popular_movies.merge(movies[['movieId','title']],how = 'left').head(10)

,movieId,mean,count,popularity_score,title
0,356,4.164134,329,247.791033,Forrest Gump (1994)
1,318,4.429022,317,238.857256,"Shawshank Redemption, The (1994)"
2,296,4.197068,307,231.299267,Pulp Fiction (1994)
3,593,4.161290,279,210.290323,"Silence of the Lambs, The (1991)"
4,2571,4.192446,278,209.548112,"Matrix, The (1999)"
5,260,4.231076,251,189.307769,Star Wars: Episode IV - A New Hope (1977)
6,480,3.750000,238,179.437500,Jurassic Park (1993)
7,110,4.031646,237,178.757911,Braveheart (1995)
8,589,3.970982,224,168.992746,Terminator 2: Judgment Day (1991)
9,527,4.225000,220,166.056250,Schindler's List (1993)


Normalize observations & give weight to decide imapact of feature

In [17]:
#Normalize observations & use weight
test = ratings.groupby('movieId').agg({'rating':'mean','userId':'count'})

In [18]:
#normalize rating
test ['rating_norm'] = (test['rating'] - test['rating'].min()) / (test['rating'].max() -test['rating'].min())

In [19]:
#normalized count
test['count_norm'] = (test['userId'] - test ['userId'].min()) / (test['userId'].max() - test['userId'].min())

In [20]:
# decide weight of feature
rating_weight = 0.75
count_weight = 0.25

In [22]:
#normalized & weighted rating & count

test ['rating_weighted']= test['rating_norm'] * rating_weight
test ['count_weighted'] = test ['count_norm'] * count_weight

In [25]:
# final rating - decides popularity
test['final_rating'] = test['rating_weighted'] + test['count_weighted']
#fianl test
test = test.sort_values('final_rating',ascending= False).reset_index()

# get movie title with high rating
top_n = movies[['movieId','title']].merge(test[['movieId','final_rating']],how = 'right')

In [26]:
# top movies
top_n.head(10)

,movieId,title,final_rating
0,318,"Shawshank Redemption, The (1994)",0.895691
1,356,Forrest Gump (1994),0.860689
2,296,Pulp Fiction (1994),0.849410
3,2571,"Matrix, The (1999)",0.826536
4,593,"Silence of the Lambs, The (1991)",0.822105
5,260,Star Wars: Episode IV - A New Hope (1977),0.812395
6,2959,Fight Club (1999),0.794219
7,527,Schindler's List (1993),0.787754
8,1196,Star Wars: Episode V - The Empire Strikes Back...,0.779334
9,50,"Usual Suspects, The (1995)",0.777683


**Top 10 movies popularity based**

In [27]:
def top_n_pop_based(n):

    # top_n is : normalizes data and different weight(2)
    top_n_movies = top_n.head(n)

    return top_n_movies

In [28]:
top_n_pop_based(10)

,movieId,title,final_rating
0,318,"Shawshank Redemption, The (1994)",0.895691
1,356,Forrest Gump (1994),0.860689
2,296,Pulp Fiction (1994),0.849410
3,2571,"Matrix, The (1999)",0.826536
4,593,"Silence of the Lambs, The (1991)",0.822105
5,260,Star Wars: Episode IV - A New Hope (1977),0.812395
6,2959,Fight Club (1999),0.794219
7,527,Schindler's List (1993),0.787754
8,1196,Star Wars: Episode V - The Empire Strikes Back...,0.779334
9,50,"Usual Suspects, The (1995)",0.777683
